In [ ]:
import argparse
import os
import time

import torch

import pytorch_mask_rcnn as pmr
    
    
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
    cuda = device.type == "cuda"
    if cuda: pmr.get_gpu_prop(show=True)
    print("\ndevice: {}".format(device))
    
    d_test = pmr.datasets(args.dataset, args.data_dir, "val", train=True) # VOC 2012. set train=True for eval
    #d_test = pmr.datasets(args.dataset, args.data_dir, "val2017", train=True) # COCO 2017

    print(args)
    num_classes = max(d_test.classes) + 1
    model = pmr.maskrcnn_resnet50(False, num_classes).to(device)
    
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    model.load_state_dict(checkpoint["model"])
    #print(checkpoint["eval_info"])
    del checkpoint
    if cuda: torch.cuda.empty_cache()

    print("\nevaluating...\n")
    
    B = time.time()
    eval_output, iter_eval = pmr.evaluate(model, d_test, device, args)
    B = time.time() - B
    
    print(eval_output.get_AP())
    if iter_eval is not None:
        print("\nTotal time of this evaluation: {:.1f} s, speed: {:.1f} imgs/s".format(B, 1 / iter_eval))
    
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", default="voc")
    parser.add_argument("--data-dir", default="E:/PyTorch/data/voc2012")
    parser.add_argument("--ckpt-path", default="E:/PyTorch/scripts/ckpts/maskrcnn_voc-5.pth")
    parser.add_argument("--iters", type=int, default=3) # number of iterations, minus means the entire dataset
    args = parser.parse_args([]) # [] is needed if you're using Jupyter Notebook.
    
    args.use_cuda = True
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "maskrcnn_results.pth")
    
    main(args)
    
   

In [12]:
 
import argparse
import os
import time

import torch

import pytorch_mask_rcnn as pmr
import torch
import torch.onnx as onnx
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image   
    
def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and args.use_cuda else "cpu")
    cuda = device.type == "cuda"
    if cuda: pmr.get_gpu_prop(show=True)
    print("\ndevice: {}".format(device))
    
    # d_test = pmr.datasets(args.dataset, args.data_dir, "val", train=True) # VOC 2012. set train=True for eval
    d_test = pmr.datasets(args.dataset, args.data_dir, "val2017", train=True) # COCO 2017

    print(args)
    num_classes = max(d_test.classes) + 1
    model = pmr.maskrcnn_resnet50(False, num_classes).to(device)
    
    checkpoint = torch.load(args.ckpt_path, map_location=device)
    model.load_state_dict(checkpoint["model"])
    #print(checkpoint["eval_info"])

    # Load a sample image from your dataset to create a dummy input
    sample_image, _ = d_test[0]
    sample_input = sample_image.to(device)
    print(sample_input.size())
    # Step 2: Define dummy input (replace this with your actual input dimensions)
    # dummy_input = torch.randn(3, 224, 224)#.to(device)
    sample_input_int = sample_input.float()

    model = model.requires_grad_(False)


    # Step 3: Export the PyTorch model to ONNX format
    onnx_path = "model.onnx"
    torch.onnx.export(model, sample_input_int, onnx_path, verbose=True, input_names=["input"], output_names=["output"], opset_version=11)

    # Step 4: Load the ONNX model
    onnx_model = onnx.load(onnx_path)

    del checkpoint
    if cuda: torch.cuda.empty_cache()

    print("\nevaluating...\n")
    
    B = time.time()
    eval_output, iter_eval = pmr.evaluate(model, d_test, device, args)
    B = time.time() - B
    
    print(eval_output.get_AP())
    if iter_eval is not None:
        print("\nTotal time of this evaluation: {:.1f} s, speed: {:.1f} imgs/s".format(B, 1 / iter_eval))
    
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", default="coco")
    parser.add_argument("--data-dir", default="/home/moka/Akash/PyTorch-Simple-MaskRCNN/data")
    parser.add_argument("--ckpt-path", default="/home/moka/Akash/PyTorch-Simple-MaskRCNN/maskrcnn_coco-3.pth")
    parser.add_argument("--iters", type=int, default=3) # number of iterations, minus means the entire dataset
    args = parser.parse_args([]) # [] is needed if you're using Jupyter Notebook.
    
    args.use_cuda = True
    args.results = os.path.join(os.path.dirname(args.ckpt_path), "maskrcnn_results.pth")
    
    main(args)
    
    


device: cpu
loading annotations into memory...


Done (t=5.60s)
creating index...
index created!
Namespace(ckpt_path='/home/moka/Akash/PyTorch-Simple-MaskRCNN/maskrcnn_coco-3.pth', data_dir='/home/moka/Akash/PyTorch-Simple-MaskRCNN/data', dataset='coco', iters=3, results='/home/moka/Akash/PyTorch-Simple-MaskRCNN/maskrcnn_results.pth', use_cuda=True)
torch.Size([3, 426, 640])


KeyboardInterrupt: 

In [5]:
import torch
import torch.onnx as onnx
import torchvision.models as models

def convert_to_onnx(ckpt_path, input_size, output_onnx_path):
    # Load the PyTorch model from the .pth file
    model = torch.load(ckpt_path)
    
    # Set the model to evaluation mode
    # model.eval()

    # Create dummy input tensor with the specified size
    dummy_input = torch.randn(input_size)

    # Export the PyTorch model to ONNX format
    onnx.export(model, dummy_input, output_onnx_path, verbose=True, input_names=["input"], output_names=["output"])

if __name__ == "__main__":
    # Specify the path to the .pth file
    ckpt_path = "/home/moka/Akash/PyTorch-Simple-MaskRCNN/maskrcnn_coco-3.pth"

    # Specify the input size (replace this with the actual input size of your model)
    input_size = (3, 224, 224)

    # Specify the output path for the ONNX file
    output_onnx_path = "model.onnx"

    # Convert the model to ONNX format
    convert_to_onnx(ckpt_path, input_size, output_onnx_path)


AttributeError: 'dict' object has no attribute 'training'